# Checking the OA status of an author's papers

In [1]:
import json
import orcid
import requests
import sys
import time
from IPython.display import HTML, display

ORCID of the author you want to check

In [2]:
ORCID = '0000-0001-5318-3910'

Sadly, to make requests to ORCID and Base APIs, you will need to register with them, I cannot share my identifiers

In [3]:
orcid_api_id = 'APP-XXXXXXXXXXXXXXXX'
orcid_api_secret = 'XXXXXXXX-XXXX-XXXX-XXXX-XXXXXXXXXXXX'
base_api_ua = 'XXXXXXXXXX@XXXXXXXXXXXXXXXXXXXXXX'
orcid_api_email = 'XXXXXXXXX@XXXXXXXXX'

Now we retrieve the papers for this author:

In [4]:
def findDOI(paper):
    for i in paper['external-ids']['external-id']:
        if i['external-id-type'] == 'doi':
            return i['external-id-value']
    return None

api = orcid.PublicAPI(orcid_api_id, orcid_api_secret)
token = api.get_search_token_from_orcid()
record = api.read_record_public(ORCID, 'record', token)
print(f'ORCID: {record["orcid-identifier"]["path"]}')
name = record['person']['name']
print(f'Name: {name["given-names"]["value"]} {name["family-name"]["value"]}')
works = api.read_record_public(ORCID, 'works', token)['group']
print(f'Number of papers in ORCID: {len(works)}\n')
      
dois = []
for paper in works:
    doi = findDOI(paper)
    if doi:
        dois.append((doi, paper['work-summary'][0]['title']['title']['value']))
    else:
        title = paper['work-summary'][0]['title']['title']['value']
        print(f'No DOI available for paper: {title}')

print(f'\nNumber of papers with DOI: {len(dois)}')

ORCID: 0000-0001-5318-3910
Name: François-Xavier Coudert
Number of papers in ORCID: 109

No DOI available for paper: Free energy landscapes for the thermodynamic understanding of adsorption-induced deformations and structural transitions in porous materials.
No DOI available for paper: Distribution of sodium cations in faujasite-type zeolite: A canonical parallel tempering simulation study
No DOI available for paper: Theoretical study of neutral dipolar atom in water: Structure, spectroscopy and formation of an excitonic state

Number of papers with DOI: 106


In [5]:
def base_search(doi):
    r = requests.get('https://api.base-search.net/cgi-bin/BaseHttpSearchInterface.fcgi',
                     params={'func': 'PerformSearch', 'query': f'dcdoi:{doi}', 'boost': 'oa', 'format': 'json'},
                     headers={'user-agent': 'fx.coudert@chimieparistech.psl.eu'})
    docs = r.json()['response']['docs']
    for doc in docs:
        if doc['dcoa'] == 1:
            return True
    return False

def unpaywall(doi):
    r = requests.get(f'https://api.unpaywall.org/v2/{doi}',
                     params={"email": "fxcoudert@gmail.com"})
    r = r.json()
    if 'error' in r:
        return False
    if r['is_oa']:
        return True
    return False

In [6]:
res = []
print('Be patient, this step takes time (2 to 3 seconds per paper)')
for doi, title in dois:
    res.append((doi, title, base_search(doi), unpaywall(doi)))
    time.sleep(1)

Be patient, this step takes time (2 to 3 seconds per paper)


In [8]:
def YesNoCell(b):
    if b:
        return '<td style="background-color: #60FF60">Yes</td>'
    else:
        return '<td style="background-color: #FF6060">No</td>'


s = ''
tot_base = 0
tot_unpaywall = 0
for doi, title, base, unpaywall in res:
    s += f'<tr><td><a href="https://doi.org/{doi}" target="_blank">{title}</a></td>{YesNoCell(base)}{YesNoCell(unpaywall)}</td>'
    if base:
        tot_base += 1
    if unpaywall:
        tot_unpaywall += 1

s += f'<tr><td>Total</td><td>{100 * tot_base/ len(res):.1f}%</td><td>{100 * tot_unpaywall/ len(res):.1f}%</td></tr>'
    
header = '<tr><td>Paper title</td><td>OA in BASE</td><td>OA in Unpaywall</td></tr>'
display(HTML('<table>' + header + s + '</table>'))

Paper title,OA in BASE,OA in Unpaywall
Rotational Dynamics of Linkers in Metal–Organic Frameworks,Yes,Yes
Ab Initio Derived Force Fields for Zeolitic Imidazolate Frameworks: MOF-FF for ZIFs,No,Yes
Charting a course for chemistry,No,No
Nanoscale metamaterials: Meta-MOFs and framework materials with anomalous behavior,No,No
Pressure promoted low-temperature melting of metal–organic frameworks,No,Yes
Adsorption Contraction Mechanics: Understanding Breathing Energetics in Isoreticular Metal-Organic Frameworks,No,No
Air separation with graphene mediated by nanowindow-rim concerted motion,Yes,Yes
Conformational chiral polymorphism in cis-bis-triphenylphosphine complexes of transition metals,No,Yes
Emissive Azobenzenes Delivered on a Silver Coordination Polymer,No,No
"Impacts of the Imidazolate Linker Substitution (CH3, Cl, or Br) on the Structural and Adsorptive Properties of ZIF-8",No,No
